In [ ]:
import numpy as np
import pandas as pd
import re # Used for the conversion of "r##_c##" in only the numbers --> TODO: check where it comes from
from IPython.display import display
from helpers import *
from play_with_data import *
from pre_processing import *
from matrix_factorization import *
from cross_validation import *
from apply_classifiers import *
from trainings_submissions import *
from regressions_models import *
import scipy.sparse as sp # In order to use sparse 
# Predictors imported in performance order (best to worst, according to http://surpriselib.com/)
from surprise import SVDpp
from surprise import KNNBaseline
from surprise import SVD
from surprise import SlopeOne
from surprise import BaselineOnly
from surprise import KNNWithMeans
from surprise import NMF
from surprise import CoClustering
from surprise import KNNBasic
from surprise import KNNWithZScore # not scored --> to be tested quickly
from surprise import dataset
from surprise import Dataset
from surprise import evaluate, print_perf
from surprise import GridSearch
from surprise import accuracy
from sklearn.metrics import mean_squared_error
from sklearn import linear_model

from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

%load_ext autoreload
%autoreload 2

In [ ]:
#******** Creation of a sparse matrix of the data **********
ratings = load_data('./data_train.csv')

In [ ]:
test = load_data('./sample_submission.csv')

In [ ]:
#*********** Formating the data correctly for Surprise + Cross Validation *****
ratings_surpr = formating_data_surprise(ratings) # Call a method that will transform the ratings in the right format
ratings_surpr.split(n_folds=3) # Will create the 3 folds for cross validation

# Hyperparameters optimization

In [ ]:
########### Define: algo, dataset (trainset ##############
ratings_hyper = formating_data_surprise(ratings)
#trainset_hyper = ratings_hyper.build_full_trainset()

In [ ]:
ratings_hyper.split(n_folds=7)

### Hyperparameters of SVDpp

In [ ]:
list_n_factors_pp = np.linspace(10,30,3)
list_lr_all_pp = np.logspace(-3.5,-2,3)
list_reg_all_pp = np.logspace(-3,-1.5,3)

In [ ]:
best_n_factors_pp = 0
best_lr_all_pp = 0
best_reg_all_pp = 0
best_rmse_pp = 1000

for n_factors in list_n_factors_pp:
    for lr_all in list_lr_all_pp:
        for reg_all in list_reg_all_pp:
            n_factors = int(n_factors)
            print(n_factors,lr_all,reg_all)

            algorithm_bsl = SVDpp(n_factors=n_factors,lr_all=lr_all,reg_all=reg_all)


            for trainset1, testset1 in ratings_hyper.folds():

                # train and test algorithm.
                algorithm_bsl.train(trainset1)
                predictions1 = algorithm_bsl.test(testset1)

                # Compute and print Root Mean Squared Error
                rmse = accuracy.rmse(predictions1, verbose=True)
                break
            if rmse < best_rmse_svd:
                print("Improvement!")
                best_n_factors_pp = n_factors
                best_lr_all_pp = lr_all
                best_reg_all_pp = reg_all
                best_rmse_pp = rmse


In [ ]:
print(best_n_factors_pp,best_lr_all_pp,best_reg_all_pp,best_rmse_pp)

### Hyperparameters of NMF

In [ ]:
list_n_factors_nmf = np.linspace(30,40,3)
list_reg_pu = np.logspace(-1.5,-1,3)
list_reg_qi = np.logspace(-1,-0.5,3)

In [ ]:
best_n_factors_nmf = 0
best_reg_pu = 0
best_reg_qi = 0
best_rmse_nmf = 1000

for n_factors in list_n_factors_nmf:
    for reg_pu in list_reg_pu:
        for reg_qi in list_reg_qi:
            n_factors = int(n_factors)
            print(n_factors,reg_pu,reg_qi)

            algorithm_bsl = NMF(n_factors=n_factors,reg_pu=reg_pu,reg_qi=reg_qi)


            for trainset1, testset1 in ratings_hyper.folds():

                # train and test algorithm.
                algorithm_bsl.train(trainset1)
                predictions1 = algorithm_bsl.test(testset1)

                # Compute and print Root Mean Squared Error
                rmse = accuracy.rmse(predictions1, verbose=True)
                break
            if rmse < best_rmse_nmf:
                print("Improvement!")
                best_n_factors_nmf = n_factors
                best_reg_pu = reg_pu
                best_reg_qi = reg_qi
                best_rmse_nmf = rmse


In [ ]:
print(best_n_factors_nmf,best_reg_pu,best_reg_qi,best_rmse_nmf)

35 0.0316227766017 0.316227766017 1.00321306113

## Hyperparameters of BaselineOnly

In [ ]:
# BaselineOnly(bsl_options={'method': 'als', 'reg_u': 14.4, 'reg_i': 0.3})

# Comparer avec
# BaselineOnly()

## Hyperparameters of KNNWithZScore

In [ ]:
# KNNWithZScore(k=100, min_k=7, sim_options={'name':'pearson_baseline','user_based':False,'shrinkage':500})

# Comparer avec
# KNNWithZScore(sim_options={'user_based':False})
# KNNWithZScore(k=100, min_k=7,sim_options={'user_based':False})

# et éventuellement, mais moins important
# KNNWithZScore(sim_options={'name':'pearson_baseline','user_based':False,'shrinkage':500})



## Hyperparameters of KNNBaseline

In [ ]:
# KNNBaseline(k=96, min_k=8,sim_options={'name': 'pearson_baseline','user_based': False,'shrinkage': 500},bsl_options={'method': 'als','reg_u': 14.4,'reg_i': 0.3})

# Comparer avec
# KNNBaseline()
# KNNBaseline(k=96, min_k=8)
# KNNBaseline(k=96, min_k=8,sim_options={'user_based':False},bsl_options={'method': 'als','reg_u': 14.4,'reg_i': 0.3})
# KNNBaseline(k=96, min_k=8,sim_options={'name': 'pearson_baseline','user_based': False,'shrinkage': 500})



## Hyperparameters of SVD

In [ ]:
# SVD(n_factors=100,lr_all=0.001,reg_all=0.0316227766017)
# SVD(n_factors=100,lr_all=0.001,reg_all=10**(-1.5))

# Comparer avec 
# SVD()

# SVD(n_factors=100,lr_all=0.001,reg_all=0.0316227766017)
# SVD(n_factors=100,lr_all=0.001,reg_all=10**(-1.5))

# Comparer avec 
# SVD()

algos = [SVD(n_factors=100,lr_all=0.001,reg_all=0.0316227766017),SVD(n_factors=100,lr_all=0.001,reg_all=10**(-1.5))\
        ,SVD()]
perf = {}
algo_str = ['SVD1','SVD2','SVD3']  

for i,algo in enumerate(algos): # Loop over the algorithms 
    # Evaluate performances of "Surprise" algorithm on the dataset
    perf[algo_str[i]] = evaluate(algo, ratings_surpr, measures=['RMSE']) # Evaluate the performance of each algo by cross validation 
    print_perf(perf[algo_str[i]]) # Print the performance for each algo

## Hyperparameters of SVDpp

In [ ]:
# A terminer encore, cf optimisation chez Stefan
# SVDpp(n_factors=n_factors,lr_all=lr_all,reg_all=reg_all)

# Comparer avec 
# SVDpp()


algos = [SVDpp(n_factors=12,lr_all=0.001,reg_all=0.000501187233627),SVDpp(n_factors=10,lr_all=0.00177827941004,reg_all=0.001)\
        ,SVDpp()]
perf = {}
algo_str = ['SVDpp1','SVDpp2','SVDpp3']  

for i,algo in enumerate(algos): # Loop over the algorithms 
    # Evaluate performances of "Surprise" algorithm on the dataset
    perf[algo_str[i]] = evaluate(algo, ratings_surpr, measures=['RMSE']) # Evaluate the performance of each algo by cross validation 
    print_perf(perf[algo_str[i]]) # Print the performance for each algo

## Hyperparameters of NMF

In [ ]:
# NMF(n_factors=35,reg_pu=0.0316227766017,reg_qi=0.316227766017)
# NMF(n_factors=35,reg_pu=10**(-1.5),reg_qi=10**(-0.5))

# Comparer avec 
# NMF()